In [1]:
from CGM_processor import CGMDataProcessor

CGM_data = CGMDataProcessor("CGM_data.csv", min_range=68, max_range=180, target=100)

CGM_data.plot_data()

In [3]:
from insulin_activity_processor import IADataProcessor
import plotly.express as px

IA_function = IADataProcessor(CGM_file_path="CGM_data.csv", bolus_file_path="Bolus_data.csv", tp=75, td=300)

fig = px.line(x=IA_function.CGM_data["Device Time"], y=IA_function.insulin_activity_curve())

fig.show()

print(IA_function.insulin_activity_curve())

[1.65741833e+04 1.65691833e+04 1.65641833e+04 ... 1.00000000e+01
 5.00000000e+00 0.00000000e+00]
[16459.73333333 16452.7        16284.38333333 15866.95
 15809.5        15694.96666667 15574.58333333 15455.01666667
 15414.63333333 15384.36666667 15124.65       15092.26666667
 14823.13333333 14326.65       14228.81666667 14200.08333333
 14147.1        13979.96666667 13889.5        13653.91666667
 13620.45       13615.18333333 13546.81666667 13518.06666667
 13478.35       13289.15       13194.76666667 12849.21666667
 12825.4        12762.4        12722.25       12673.15
 12669.4        12601.56666667 12576.26666667 12561.96666667
 12496.7        12140.08333333 12101.8        12001.83333333
 11861.93333333 11515.61666667 11501.28333333 11476.46666667
 11448.28333333 11324.01666667 11224.96666667 11201.73333333
 11177.68333333 11112.78333333 11047.9        10981.
 10935.01666667 10676.81666667 10667.48333333 10522.1
 10502.63333333  9836.08333333  9660.7         9635.01666667
  9463.96666667

[1.65741833e+04 1.65691833e+04 1.65641833e+04 ... 1.00000000e+01
 5.00000000e+00 0.00000000e+00]
[16459.73333333 16452.7        16284.38333333 15866.95
 15809.5        15694.96666667 15574.58333333 15455.01666667
 15414.63333333 15384.36666667 15124.65       15092.26666667
 14823.13333333 14326.65       14228.81666667 14200.08333333
 14147.1        13979.96666667 13889.5        13653.91666667
 13620.45       13615.18333333 13546.81666667 13518.06666667
 13478.35       13289.15       13194.76666667 12849.21666667
 12825.4        12762.4        12722.25       12673.15
 12669.4        12601.56666667 12576.26666667 12561.96666667
 12496.7        12140.08333333 12101.8        12001.83333333
 11861.93333333 11515.61666667 11501.28333333 11476.46666667
 11448.28333333 11324.01666667 11224.96666667 11201.73333333
 11177.68333333 11112.78333333 11047.9        10981.
 10935.01666667 10676.81666667 10667.48333333 10522.1
 10502.63333333  9836.08333333  9660.7         9635.01666667
  9463.96666667